In [1]:
# Only for Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch

import numpy as np
import torch.nn.functional as nnF
import torchtext.functional as F
import torch.nn as nn
import torchtext.transforms as T

from torch.autograd import Variable
from torch.hub import load_state_dict_from_url
from torchtext.vocab import GloVe


class NN_Predictor:
  def __init__(self, model_path):
    self.model_path = model_path
    self.padding_idx = 1
    self.bos_idx = 0
    self.eos_idx = 2
    self.max_seq_len = 256
    self.xlmr_vocab_path = r"https://download.pytorch.org/models/text/xlmr.vocab.pt"
    self.xlmr_spm_model_path = r"https://download.pytorch.org/models/text/xlmr.sentencepiece.bpe.model"

    self.text_transform = T.Sequential(
        T.SentencePieceTokenizer(self.xlmr_spm_model_path),
        T.VocabTransform(load_state_dict_from_url(self.xlmr_vocab_path)),
        T.Truncate(self.max_seq_len - 2),
        T.AddToken(token=self.bos_idx, begin=True),
        T.AddToken(token=self.eos_idx, begin=False),
    )
    self.vocab = self.text_transform[1].vocab.vocab
    self.word_to_idx = self.vocab.get_stoi()

  def maybe_gpu(self, v, gpu):
      return v.cuda() if gpu else v

  def get_pretrained_embeddings(self, hp):
    glove_vectors = GloVe(name="6B", dim=hp['EMBEDDING_DIM'])
    EMBEDDING_DIM = glove_vectors.vectors.shape[1]
    pretrained_embeddings = np.random.uniform(-0.25, 0.25, (len(self.vocab), EMBEDDING_DIM)).astype('f')
    pretrained_embeddings[0] = 0
    for word, wi in glove_vectors.stoi.items():
        try:
            pretrained_embeddings[self.word_to_idx[word]-1] = glove_vectors.__getitem__(word)
        except KeyError:
            pass
    pretrained_embeddings = self.maybe_gpu(torch.from_numpy(pretrained_embeddings), hp['USE_GPU'])
    return pretrained_embeddings

  def get_switcher_model(parent_self):
    class Categorizer(nn.Module):
      def __init__(self, embedding_dim, vocab_size, label_size, use_gpu):
          """Prepare individual layers"""
          super(Categorizer, self).__init__()
          self.use_gpu = use_gpu
          self.embeddings = nn.Embedding(vocab_size, embedding_dim)
          self.network = nn.Sequential(
              nn.Linear(embedding_dim, 100),
              nn.ReLU(),
              nn.Linear(100, 50),
              nn.ReLU(),
              nn.Linear(50, label_size)
          )

      def forward(self, sentence):
          """Use the layers of this model to propagate input and return class log probabilities"""
          if self.use_gpu:
              sentence = sentence.cuda()
          x = self.embeddings(sentence).permute(1, 0, 2)
          x = x.mean(dim=0)
          output = self.network(x)
          return output
    return Categorizer

  def prepare_model(self, Categorizer, hp, pretrained_embeddings):
    num_classes = 5
    categorizer = Categorizer(embedding_dim=hp["EMBEDDING_DIM"], vocab_size=len(self.vocab), label_size=num_classes, use_gpu=hp["USE_GPU"])
    categorizer.embeddings = nn.Embedding.from_pretrained(pretrained_embeddings)
    model = categorizer
    return model

  def load_model(self):
    hp = {
        "EPOCHS": 10,
        "BATCH_SIZE": 16,
        "LEARNING_RATE": 1e-2,
        "EMBEDDING_DIM": 50,
        "USE_GPU": torch.cuda.is_available()
    }
    pretrained_embeddings = self.get_pretrained_embeddings(hp)
    Categorizer = self.get_switcher_model()
    self.model = self.prepare_model(Categorizer, hp, pretrained_embeddings)
    self.model.load_state_dict(torch.load(self.model_path, map_location=torch.device('cpu')))

  def get_prediction(self, text):
    with torch.no_grad():
      classes = ['audio_to_text_data', 'blog_scrapping', 'random_data', 'resume_summary_data', 'suduko_data']
      self.model.eval()
      text = self.text_transform([text])
      text_tensor = F.to_tensor(text, self.padding_idx)
      prediction = self.model(text_tensor)
      probs = torch.softmax(prediction, dim=-1)[0].tolist()
      final_prediction = list(zip(classes, probs))
      final_prediction = sorted(final_prediction, key = lambda x: -x[1])
      return final_prediction

In [7]:
# Initialize Predictor only once at server start up
MODEL_PATH = 'drive/MyDrive/NN.pt'
predictor = NN_Predictor(MODEL_PATH)
predictor.load_model()

In [8]:
# Run prediction for each API call
predictor.get_prediction('Solve this Sudoku puzzle for me')

[('suduko_data', 0.9999984502792358),
 ('audio_to_text_data', 9.453690950067539e-07),
 ('blog_scrapping', 5.991369675939495e-07),
 ('random_data', 9.271002854305266e-10),
 ('resume_summary_data', 4.996553726321906e-10)]